In [ ]:
! pip install lightfm

In [ ]:
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
import numpy as np

In [ ]:
# Завантаження даних
data = pd.read_csv("ratings.csv")

# Підготовка даних
dataset = Dataset()
dataset.fit(data["userId"], data["movieId"])

# Побудова взаємодій (інтеракцій) матриці
(interactions, weights) = dataset.build_interactions(
    ((row["userId"], row["movieId"], row["rating"]) for index, row in data.iterrows())
)

# Побудова моделі
model = LightFM(loss="warp")
model.fit(interactions, epochs=30, num_threads=2)

# Оцінка моделі
train_precision = precision_at_k(model, interactions, k=10).mean()
train_recall = recall_at_k(model, interactions, k=10).mean()

print(f"Train precision: {train_precision}")
print(f"Train recall: {train_recall}")

# Рекомендації для конкретного користувача
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data.shape
    
    for user_id in user_ids:
        scores = model.predict(user_id, np.arange(n_items))
        top_items = np.argsort(-scores)[:10]
        
        print(f"Recommendations for user {user_id}:")
        for x in top_items:
            print(f"    {x}")

sample_recommendation(model, interactions, [3, 25, 450])
